In [2]:
from general_tools.notebook import gpu_utils
GPU = 2
gpu_utils.setup_one_gpu(GPU)

Picking GPU 2


In [44]:
import sys
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.svm import LinearSVC

import tf_lab.point_clouds.in_out as pio

from tf_lab.point_clouds.in_out import load_point_clouds_from_filenames, PointCloudDataSet
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf
from tf_lab.in_out.basics import read_saved_epochs

from tf_lab.data_sets.model_net import pc_loader, classes_to_integers
from tf_lab.data_sets.pc_utils import pclouds_with_zero_mean_in_unit_sphere


from geo_tool import Point_Cloud

from general_tools.notebook.tf import reset_tf_graph
from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs
from general_tools.classification.evaluations import average_per_class
from general_tools.simpletons import sort_dict_by_key, sort_dict_by_val

In [4]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:
def iclr_optimized_paratemers(loss, model_net, dual=False):    
    if loss == 'chamfer':
        epoch_to_load = 1000
        if model_net == '40': # ignores dual
            svm_params = (0.25, 0.4, False, 'squared_hinge')     # Acc = 84.5
        elif model_net == '10':
            if dual:
                svm_params = (0.2, 3, True, 'hinge')             # Max Acc = 96.1
            else:
                svm_params = (0.05, 0.2, False, 'squared_hinge') # Acc = 95.40
    
    if loss == 'emd':
        epoch_to_load = 1100
        if model_net == '40':
            if dual:
                svm_params = (0.09, 0.5, True, 'hinge')          # Max Acc = 84.1, (stable)
            else:
                svm_params = (0.07, 3, False, 'squared_hinge')   # Acc = 83.7

        elif model_net == '10':
            if dual:
                svm_params = (0.15, 0.1, True, 'hinge')          # Max Acc = 95.9
            else:
                svm_params = (0.02, 3, False, 'squared_hinge')   # Acc = 95.46


    return svm_params, epoch_to_load    

In [6]:
n_pc_samples = 2048
put_in_usphere = True
model_net = '40'
ae_loss = 'chamfer'
do_verification = True
solve_dual = True

In [7]:
top_data_dir = '/orions4-zfs/projects/optas/DATA/OUT/iclr/nn_models/all_snc/'
experiment_name = 'no_conv_deeper_snc_rotated2048pts_' + ae_loss

In [48]:
train_dir = osp.join(top_data_dir, experiment_name)
conf = Conf.load(osp.join(train_dir, 'configuration'))

## NEW for adaptable b_norm parameter is encoder_with_conv_symmetry 
# conf.encoder_args['b_norm'] = [conf.encoder_args['b_norm']]
# conf.decoder_args['b_norm'] = [conf.decoder_args['b_norm']]
##

if conf.train_dir != train_dir: # added to address moving training-folder to other location
    conf.train_dir = train_dir
    conf.save(osp.join(conf.train_dir, 'configuration'))

print conf
conf.n_output = conf.n_input
conf.allow_gpu_growth = True

              allow_gpu_growth: False
                    batch_size: 50
                 consistent_io: None
                         debug: False
                       decoder: decoder_with_fc_only
                  decoder_args: {'b_norm': True, 'layer_sizes': [1024, 2048, 6144]}
                       encoder: encoder_with_convs_and_symmetry_new
                  encoder_args: {'filter_sizes': [1, 1, 1, 1], 'n_filters': [128, 128, 256, 512], 'b_norm': True, 'strides': [1, 1, 1, 1]}
               experiment_name: no_conv_deeper_snc_rotated2048pts_chamfer
                 gauss_augment: None
                  is_denoising: False
               latent_vs_recon: 1.0
                 learning_rate: 0.0005
                          loss: chamfer
             loss_display_step: 1
                       n_input: [2048, 3]
                      n_output: [2048, 3]
                           n_z: None
             saver_max_to_keep: None
                    saver_step: 10
                 

In [33]:
(c, intercept, dual, svm_loss), epoch_to_load = iclr_optimized_paratemers(ae_loss, model_net, solve_dual)

In [49]:
reset_tf_graph()
ae = PointNetAutoEncoder(experiment_name, conf)
saved_epochs = read_saved_epochs(conf.train_dir)
last_epoch = saved_epochs[-1]
if do_verification:
    last_epoch = epoch_to_load
ae.restore_model(conf.train_dir, last_epoch, verbose=True)

TypeError: object of type 'bool' has no len()

In [35]:
model_net_dir = '/orions4-zfs/projects/optas/DATA/Point_Clouds/Model_Net_' + model_net + '/from_manifold/'
model_net_dir = osp.join(model_net_dir, str(n_pc_samples))

search_pattern = '(.*)train(.*)\.ply$'
train_pc_files = sorted([f for f in files_in_subdirs(model_net_dir, search_pattern)])

search_pattern = '(.*)test(.*)\.ply$'
test_pc_files = sorted([f for f in files_in_subdirs(model_net_dir, search_pattern)])

In [36]:
verbose = True
n_threads = 22

pc, model_names, labels = load_point_clouds_from_filenames(train_pc_files, n_threads, pc_loader, verbose)

if put_in_usphere:
    pc = pclouds_with_zero_mean_in_unit_sphere(pc)

train_data = PointCloudDataSet(pc, labels=labels, init_shuffle=False)

pc_, model_names_, labels_ = load_point_clouds_from_filenames(test_pc_files, n_threads, pc_loader, verbose)

if put_in_usphere:
    pc_ = pclouds_with_zero_mean_in_unit_sphere(pc_)

test_data = PointCloudDataSet(pc_, labels=labels_, init_shuffle=False)

9843 pclouds were loaded. They belong in 40 shape-classes.
2468 pclouds were loaded. They belong in 40 shape-classes.


In [37]:
train_feed, train_latent, train_classes = ae.embedding_at_tensor(train_data, conf, tensor_name='bottleneck')
cids = classes_to_integers(int(model_net), train_classes)[1]

test_feed, test_latent, test_classes = ae.embedding_at_tensor(test_data, conf, tensor_name='bottleneck')
cids_ = classes_to_integers(int(model_net), test_classes)[1]

In [43]:
if do_verification:
    print c, svm_loss, intercept, dual
    lsvc = LinearSVC(C=c, loss=svm_loss, intercept_scaling=intercept, dual=dual, random_state=42)
    lsvc.fit(train_latent, cids)                    
    y_pred = lsvc.predict(test_latent)    
    score = average_per_class(y_pred, cids_)[0]
    print score

0.8450465116279069


In [ ]:
all_scores = dict()

intercept_scaling_space = [0.1, 0.2, 0.4, 0.5, 1, 2, 3, 4]

c_space = [0.01, 0.02, 0.05, 0.07, 0.08, 0.09,
           0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 
           0.45, 0.50]

dual_space = [False, True]

loss_space = ['hinge', 'squared_hinge']

for c in c_space:
    for intercept in intercept_scaling_space:
        for dual in dual_space:
            for loss in loss_space:
                try:                    
                    lsvc = LinearSVC(C=c, loss=loss, intercept_scaling=intercept, dual=dual, random_state=42)
                    lsvc.fit(train_latent, cids)
                    print c, intercept, dual, loss
                    y_pred = lsvc.predict(test_latent)    
                    score = average_per_class(y_pred, cids_)[0]
                    print score
                    key = (c, intercept, dual, loss)
                    all_scores[key] = score
                except:
                    continue

best_responses = sort_dict_by_val(all_scores, reverse=True)            

0.01 0.1 False squared_hinge
0.94876744186
0.01 0.1 True hinge
0.936627906977
0.01 0.1 True squared_hinge
0.949930232558
0.01 0.2 False squared_hinge
0.949930232558
0.01 0.2 True hinge
0.937627906977
0.01 0.2 True squared_hinge
0.94876744186
0.01 0.4 False squared_hinge
0.949930232558
0.01 0.4 True hinge
0.937627906977
0.01 0.4 True squared_hinge
0.949930232558
0.01 0.5 False squared_hinge
0.949930232558
0.01 0.5 True squared_hinge
0.949930232558
0.01 1 False squared_hinge
0.94876744186
0.01 1 True hinge
0.937627906977


In [ ]:
# FROM NIPS
# USING ALL ShapeNetCore DATA : ~57K models.
# Training at (max) 2K epochs.

# EMD, 2K, model_net_40
# 1.0 3 False squared_hinge
# 0.844814259486
# 1.0 3 True squared_hinge
# 0.844814259486

# EMD, 2K, model_net_10
# 0.4 2 True hinge
# 0.953279069767


# Chamfer, 2K model_net_40
# 0.3 4 False squared_hinge
# 0.856074051408
# 0.3 4 True squared_hinge
# 0.856074051408
# 0.6 1 False squared_hinge
# 0.856264075887
# 0.3 4 False squared_hinge
# 0.856074051408
# 0.6 1 False squared_hinge
# 0.856264075887
# 0.6 1 True squared_hinge
# 0.856264075887
# 0.7 1 False squared_hinge
# 0.856764075887

# Chamfer, 2K model_net_10
# 0.9 4 True hinge
# 0.950790697674
# 0.950790697674
# 0.8 0.5 False squared_hinge

# 1.0 4 True hinge
# 0.950790697674
# 1.0 3 True hinge
# 0.950790697674
# 0.9 4 True hinge
# 0.950790697674

# 0.7 4 True hinge
# 0.950790697674


# With WU 7 Classes:

# M10, CD, 2K epochs
# 0.4 4 True hinge
# 0.95176744186

# M40, CD, 2K epochs
# 1.0 1 True hinge
# 0.84410495716

# M10, EMD, ~2.5K epochs
# about 0.94

# M40, EMD, ~2.5K epochs
# about 0.83

# all_flower_pots = test_feed[np.array(cids_) == 15]
# for i in range(len(all_flower_pots)):
#     Point_Cloud(all_flower_pots[i]).plot()
    
# y_pred = lsvc.predict(test_latent)
# gt_labels = np.array(cids_)

# scores_per_class = []

# for c in np.unique(gt_labels):
#     if c == 15:
#         scores_per_class.append(0.5)    
#         continue
#     index_c = gt_labels == c
#     n_class = float(np.sum(index_c))
#     s = np.sum(gt_labels[index_c] == y_pred[index_c])
#     s /= n_class
#     scores_per_class.append(s)
# print np.mean(scores_per_class)